In [3]:
import random
import re

# Historical S&P 500 returns from 1926 to 2023
total_returns = [
    11.62, 37.49, 43.61, -8.42, -24.90, -43.34, -8.19, 53.99, -1.44, 47.67,
    33.92, -35.03, 31.12, -0.41, -9.78, -11.59, 20.34, 25.90, 19.75, 36.44,
    -8.07, 5.71, 5.50, 18.79, 31.71, 24.02, 18.37, -0.99, 52.62, 31.56,
    6.56, -10.78, 43.36, 11.96, 0.47, 26.89, -8.73, 22.80, 16.48, 12.45,
    -10.06, 23.98, 11.06, -8.50, 4.01, 14.31, 18.98, -14.66, -26.47, 37.20,
    23.84, -7.18, 6.56, 18.44, 32.42, -4.91, 21.55, 22.56, 6.27, 31.73,
    18.67, 5.25, 16.61, 31.69, -3.10, 30.47, 7.62, 10.08, 1.32, 37.58,
    22.96, 33.36, 28.58, 21.04, -9.10, -11.89, -22.10, 28.68, 10.88, 4.91,
    15.79, 5.49, -37.00, 26.46, 15.06, 2.11, 16.00, 32.39, 13.69, 1.38,
    11.96, 21.83, -4.38, 31.49, 18.40, 28.71, -18.11, 26.29, -0.24
]

# Function to calculate the final return after a given number of years with varying annual returns
def calculate_return(investment, returns, years):
    final_amount = investment
    for year in range(years):
        annual_return = returns[year % len(returns)]  # Cycle through the list if fewer returns are provided
        final_amount *= (1 + annual_return / 100)
    return final_amount

# Function to calculate statistics
def calculate_statistics(results, initial_investment, target_value):
    min_result = min(results)
    avg_result = sum(results) / len(results)
    max_result = max(results)
    probability_below_initial = sum(amount <= initial_investment for amount in results) / len(results)
    probability_below_target = sum(amount <= target_value for amount in results) / len(results)
    return min_result, avg_result, max_result, probability_below_initial, probability_below_target

# Asking the user for input
initial_investment_input = input("Enter your initial investment amount: $")

# Extract numeric part of the input, handle accidental input of a letter "d"
numeric_part = re.sub('[^0-9.]', '', initial_investment_input)
try:
    initial_investment = float(numeric_part)
except ValueError:
    initial_investment = float(initial_investment_input)

years_to_hold_input = input("Enter years to hold: ")

# Handle accidental numeric input for years to hold
try:
    years_to_hold = int(years_to_hold_input)
except ValueError:
    years_to_hold = int(years_to_hold_input)

# Calculating the target value with a 3% US inflation rate
us_inflation_rate = 3.0 / 100
target_value_with_us_inflation = initial_investment * ((1 + us_inflation_rate) ** years_to_hold)

# Simulation 1: Real 20-Year Intervals
real_interval_results = []
for _ in range(1000000):
    start_year = random.randint(0, len(total_returns) - years_to_hold)
    selected_years = total_returns[start_year:start_year + years_to_hold]
    final_amount = calculate_return(initial_investment, selected_years, years_to_hold)
    real_interval_results.append(final_amount)

# Simulation 2: Combined 10-Year Intervals for 20 Years
combined_interval_years = 10
combined_interval_results = []
for _ in range(1000000):
    start_year_1 = random.randint(0, len(total_returns) - combined_interval_years)
    start_year_2 = random.randint(0, len(total_returns) - combined_interval_years)
    selected_intervals = [total_returns[start_year_1:start_year_1 + combined_interval_years],
                          total_returns[start_year_2:start_year_2 + combined_interval_years]]
    combined_years = [year for interval in selected_intervals for year in interval]
    final_amount = calculate_return(initial_investment, combined_years, years_to_hold)
    combined_interval_results.append(final_amount)

# Simulation 3: Random Intervals
random_interval_results = []
for _ in range(1000000):
    selected_years = random.sample(total_returns, years_to_hold)
    final_amount = calculate_return(initial_investment, selected_years, years_to_hold)
    random_interval_results.append(final_amount)

# Statistics for each simulation
min_real, avg_real, max_real, prob_below_initial_real, prob_below_target_real = calculate_statistics(real_interval_results, initial_investment, target_value_with_us_inflation)
min_random, avg_random, max_random, prob_below_initial_random, prob_below_target_random = calculate_statistics(random_interval_results, initial_investment, target_value_with_us_inflation)
min_combined, avg_combined, max_combined, prob_below_initial_combined, prob_below_target_combined = calculate_statistics(combined_interval_results, initial_investment, target_value_with_us_inflation)

# Printing the target value with inflation
print(f"\nValue of ${initial_investment:,.0f} if it grows by the average US rate of inflation of 3% for {years_to_hold} years = ${target_value_with_us_inflation:,.2f}")

# Printing the results for each simulation
print(f"\nReal {years_to_hold}-Year Intervals (1926-2023):")
print(f"Lowest Outcome: ${min_real:,.2f}")
print(f"Average Outcome: ${avg_real:,.2f}")
print(f"Highest Outcome: ${max_real:,.2f}")
print(f"Chance of Net Gains Lower or Equal to Initial Investment: {prob_below_initial_real:.2%}")
print(f"Chance of Net Gains Lower or Equal to ${target_value_with_us_inflation:,.2f}: {prob_below_target_real:.2%}")

print(f"\nCombined {combined_interval_years}-Year Intervals for {years_to_hold} Years (1,000,000 Simulations):")
print(f"Lowest Outcome: ${min_combined:,.2f}")
print(f"Average Outcome: ${avg_combined:,.2f}")
print(f"Highest Outcome: ${max_combined:,.2f}")
print(f"Chance of Net Gains Lower or Equal to Initial Investment: {prob_below_initial_combined:.2%}")
print(f"Chance of Net Gains Lower or Equal to ${target_value_with_us_inflation:,.2f}: {prob_below_target_combined:.2%}")

print(f"\nRandom {years_to_hold} years (1,000,000 Simulations):")
print(f"Lowest Outcome: ${min_random:,.2f}")
print(f"Average Outcome: ${avg_random:,.2f}")
print(f"Highest Outcome: ${max_random:,.2f}")
print(f"Chance of Net Gains Lower or Equal to Initial Investment: {prob_below_initial_random:.2%}")
print(f"Chance of Net Gains Lower or Equal to ${target_value_with_us_inflation:,.2f}: {prob_below_target_random:.2%}")


Enter your initial investment amount: $1000000
Enter years to hold: 20

Value of $1,000,000 if it grows by the average US rate of inflation of 3% for 20 years = $1,806,111.23

Real 20-Year Intervals (1926-2023):
Lowest Outcome: $1,844,244.85
Average Outcome: $9,145,883.93
Highest Outcome: $26,835,968.33
Chance of Net Gains Lower or Equal to Initial Investment: 0.00%
Chance of Net Gains Lower or Equal to $1,806,111.23: 0.00%

Combined 10-Year Intervals for 20 Years (1,000,000 Simulations):
Lowest Outcome: $756,691.78
Average Outcome: $9,072,029.20
Highest Outcome: $38,731,258.48
Chance of Net Gains Lower or Equal to Initial Investment: 0.31%
Chance of Net Gains Lower or Equal to $1,806,111.23: 2.84%

Random 20 years (1,000,000 Simulations):
Lowest Outcome: $204,678.38
Average Outcome: $9,142,156.55
Highest Outcome: $134,171,530.01
Chance of Net Gains Lower or Equal to Initial Investment: 0.72%
Chance of Net Gains Lower or Equal to $1,806,111.23: 4.30%
